In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

#Prueba API 2013 - 2014


In [11]:
def fetch_earthquake_data(start_date, end_date):
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime={start_date}&endtime={end_date}"
    data = pd.read_csv(url)
    return data

start_year = 2013
end_year = 2014

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"
        end_date = f"{year}-{month:02d}-31"

        data = fetch_earthquake_data(start_date, end_date)

        # Aquí puedes realizar las operaciones o análisis que necesites con los datos obtenidos
        df = pd.DataFrame(data)

In [ ]:
df

#Iterando

In [4]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?"
start_year = 2018
end_year = 2023

data_list = []

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"
        end_date = f"{year}-{month:02d}-31"
        params = {
            "format": "geojson",
            "starttime": start_date,
            "endtime": end_date,
            "limit": 20000  # Limitamos la cantidad de eventos por solicitud
        }

        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()

            for feature in data["features"]:
                properties = feature["properties"]
                magnitude = properties["mag"]
                place = properties["place"]
                date_time = pd.to_datetime(properties["time"], unit='ms')
                mag_type = properties["magType"]
                event_type = properties["type"]
                latitude = feature["geometry"]["coordinates"][1]
                longitude = feature["geometry"]["coordinates"][0]
                depth = feature["geometry"]["coordinates"][2]
                event_id = feature["id"]

                data_list.append([magnitude, place, date_time, mag_type, event_type, latitude, longitude, depth, event_id])
        else:
            print("Error en la solicitud:", response.status_code)

df = pd.DataFrame(data_list, columns=["Magnitud", "Lugar", "Fecha", "Tipo de Magnitud", "Tipo de Evento", "Latitud", "Longitud", "Profundidad", "ID"])


In [5]:
df

,Magnitud,Lugar,Fecha,Tipo de Magnitud,Tipo de Evento,Latitud,Longitud,Profundidad,ID
0,0.99,"7 km WSW of Alex, Oklahoma",2022-01-30 23:58:39.500,ml,earthquake,34.879167,-97.852000,14.67,ok2022ccvb
1,1.40,Central Alaska,2022-01-30 23:56:19.237,ml,earthquake,62.910000,-151.177400,107.60,ak0221e05fgq
2,1.10,"34 km SE of Denali National Park, Alaska",2022-01-30 23:54:44.477,ml,earthquake,63.304800,-151.272100,8.80,ak0221e051u3
3,1.80,"35 km S of Adak, Alaska",2022-01-30 23:54:32.973,ml,earthquake,51.557800,-176.673800,39.60,ak0221e050xt
4,1.10,"34 km SE of Denali National Park, Alaska",2022-01-30 23:51:44.207,ml,earthquake,63.291700,-151.321700,9.90,ak0221e04enn
...,...,...,...,...,...,...,...,...,...
221108,2.64,"44km WNW of Petrolia, CA",2023-06-01 00:25:31.000,md,earthquake,40.467500,-124.764167,23.56,nc73894866
221109,0.69,"6 km SSW of Volcano, Hawaii",2023-06-01 00:13:55.580,md,earthquake,19.386667,-155.247833,1.64,hv73433027
221110,1.80,"83 km NNW of Yakutat, Alaska",2023-06-01 00:08:19.505,ml,earthquake,60.269000,-140.128500,6.20,ak0236zdqifl
221111,5.10,"New Britain region, Papua New Guinea",2023-06-01 00:05:17.308,mb,earthquake,-5.769800,151.998300,35.00,us7000k5eb


In [18]:
# los paises y los estados (USA) que se buscan filtrar
chile = ['Chile']
japon = ['Japan']
us_estados = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
             'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
             'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
             'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
             'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
             'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
             'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
             'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
             'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [20]:
# Reemplazar valores NaN con cadena vacía
df['Lugar'] = df['Lugar'].fillna('')

# Filtrar por países y estados
df_chile = df[df['Lugar'].str.contains('|'.join(chile))]
df_japan = df[df['Lugar'].str.contains('|'.join(japon))]
df_usa = df[df['Lugar'].str.contains('|'.join(us_estados))]

In [22]:
df_japan

,Magnitud,Lugar,Fecha,Tipo de Magnitud,Tipo de Evento,Latitud,Longitud,Profundidad,ID
144,4.5,"Bonin Islands, Japan region",2022-01-30 16:41:07.413,mb,earthquake,28.7267,142.6607,10.000,us7000ghyy
326,3.9,"15 km ENE of Nara-shi, Japan",2022-01-30 06:35:56.321,ml,earthquake,34.7475,135.9508,10.000,us7000ggci
342,4.3,"off the east coast of Honshu, Japan",2022-01-30 05:22:23.246,mb,earthquake,37.8445,144.8923,10.000,us7000ghy9
431,4.0,"16 km W of Mutsu, Japan",2022-01-29 23:31:06.758,mb,earthquake,41.3159,141.0224,96.180,us7000ghvr
564,4.4,"Volcano Islands, Japan region",2022-01-29 14:44:43.071,mb,earthquake,25.0140,141.0461,185.750,us7000ghv6
...,...,...,...,...,...,...,...,...,...
219955,4.9,"25 km SW of Urakawa, Japan",2023-06-03 10:35:40.680,mb,earthquake,41.9746,142.5858,64.440,us7000k62j
220169,4.2,"73 km ESE of Katsuren-haebaru, Japan",2023-06-02 23:43:40.332,mb,earthquake,25.9991,128.5121,10.000,us7000k7ch
220494,4.1,"20 km NW of Horokanai, Japan",2023-06-02 10:08:26.242,mb,earthquake,44.1372,141.9680,216.317,us7000k5qf
220867,4.5,"Izu Islands, Japan region",2023-06-01 13:28:54.061,mb,earthquake,29.2837,142.2780,10.000,us7000k5hl
